## Testing a function to read the labJack T7-Pro
Uses 9 analog inputs (AINs) to read the data at 100 Hz.

Craig Lage - Mar 7, 2022

In [ ]:
import sys, time, datetime, asyncio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
from labjack import ljm  # Needed pip install labjack-ljm
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u
from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst_efd_client import EfdClient
from lsst.daf.butler import Butler

In [ ]:
# Open LabJack
handle = ljm.openS("T7", "wifi", "139.229.164.249")  

In [ ]:
# Verify that you are communicating
info = ljm.getHandleInfo(handle)
print("Opened a LabJack with Device type: %i, Connection type: %i,\n"
      "Serial number: %i, IP address: %s, Port: %i,\nMax bytes per MB: %i" %
      (info[0], info[1], info[2], ljm.numberToIP(info[3]), info[4], info[5]))

deviceType = info[0]
print(deviceType)

In [ ]:
# Structures to hold the calibration data and connection information
class zeroOffset:
    def __init__(self, off_x, off_y, off_z):
        # This is the reading in Volts when accel = 0
        self.x = off_x
        self.y = off_y
        self.z = off_z
        
class gMult:
    def __init__(self, mult_x, mult_y, mult_z):
        # This is the conversion to acceleration in V/g
        self.x = mult_x
        self.y = mult_y
        self.z = mult_z
        
class AIN_name:
    def __init__(self, ain_x, ain_y, ain_z):
        # This is where the sensors are connected to the labJack
        self.x = ain_x
        self.y = ain_y
        self.z = ain_z

class calData:
    def __init__(self, serial="", ain_x="", ain_y="", ain_z="", off_x=0.0, off_y=0.0, off_z=0.0, mult_x=1.0, mult_y=1.0, mult_z=1.0):
        # The serial number is imprinted on the accelerometer
        self.serial = serial
        self.AIN_name = AIN_name(ain_x, ain_y, ain_z)
        self.zeroOffset = zeroOffset(off_x, off_y, off_z)
        self.gMult = gMult(mult_x, mult_y, mult_z)
        
calDict = {}
calDict["1"] = calData(serial="A395429", ain_x="AIN1", ain_y="AIN2", ain_z="AIN3", off_x=2.49017, off_y=2.44424, off_z=2.44589, mult_x=0.98959, mult_y=0.98572, mult_z=0.99946)
calDict["2"] = calData(serial="A395423", ain_x="AIN4", ain_y="AIN5", ain_z="AIN6", off_x=2.49874, off_y=2.49595, off_z=2.41423, mult_x=0.99740, mult_y=1.00142, mult_z=0.99595)
calDict["3"] = calData(serial="A395446", ain_x="AIN7", ain_y="AIN8", ain_z="AIN9", off_x=2.47830, off_y=2.48088, off_z=2.41385, mult_x=0.97957, mult_y=0.98699, mult_z=1.00376)

In [ ]:
# Create list of AIN names and initialize

# Ensure triggered stream is disabled.
ljm.eWriteName(handle, "STREAM_TRIGGER_INDEX", 0)
# Enabling internally-clocked stream.
ljm.eWriteName(handle, "STREAM_CLOCK_SOURCE", 0)

# All negative channels are single-ended, AIN0 and AIN1 ranges are
# +/-10 V, stream settling is 0 (default) and stream resolution index
# is 0 (default).

aRange = 10.0 # +/- 10.0 Volts
aSettle = 0 # 0 microsecond settling time
resIndex = 0
aNames = ["AIN_ALL_NEGATIVE_CH", "STREAM_SETTLING_US", "STREAM_RESOLUTION_INDEX"] # List of set-up parameters
aValues = [ljm.constants.GND, aSettle, resIndex] # List of set-up values

aScanListNames = [] # List of AIN names which will be read
offsets = []
gMults = []
for name in ["1", "2", "3"]:
    for axis in ["x", "y", "z"]:
        exec(f"aName = calDict['{name}'].AIN_name.{axis}")
        aScanListNames.append(aName)
        aNames.append(aName+"_RANGE")
        aValues.append(aRange)
        exec(f"off = calDict['{name}'].zeroOffset.{axis}")
        offsets.append(off)
        exec(f"gMult = calDict['{name}'].gMult.{axis}")
        gMults.append(gMult)

offsets = np.array(offsets)
gMults = np.array(gMults)
         
# Write the analog inputs' negative channels (when applicable), ranges,
# stream settling time and stream resolution configuration.
numFrames = len(aNames)
ljm.eWriteNames(handle, numFrames, aNames, aValues)


In [ ]:
def readStream(readTime, expId, aScanListNames=aScanListNames, handle=handle, scanRate=100, \
               offsets=offsets, gMults=gMults):
    # This reads the accelerometers for a time readTime
    # and returns a Pandas timeSeries with the data
    # aScanListNames is the list of AIN ports (9 in total)
    # handle is the handle for talking to the labJack
    # scanRate is the read frequency in Hertz
    # readTime is the total time of read in seconds
    # calDict is the dictionary with the calibration data
    # The function returns a Pandas dataframe with the three 
    # accelerometers times three axes results
    
    numAddresses = len(aScanListNames)
    aScanList = ljm.namesToAddresses(numAddresses, aScanListNames)[0]
    scansPerRead = int(scanRate * readTime)
    # Configure and start stream
    scanRate = ljm.eStreamStart(handle, scansPerRead, numAddresses, aScanList, scanRate)
    start = datetime.datetime.now()
    # Stream the data
    ret = ljm.eStreamRead(handle)
    # Stop the stream
    ljm.eStreamStop(handle)
    aData = ret[0]
    # Reshape the data
    newData = np.resize(aData, (scansPerRead, numAddresses))
    # Convert to g
    accelData = (newData - offsets) / gMults
    # Create the timestamps
    end = start + datetime.timedelta(seconds = readTime)
    date_rng = pd.date_range(start=start, end=end, periods=scansPerRead)
    # Create the Pandas dataFrame
    df = pd.DataFrame(accelData, index=date_rng, 
                      columns=['ELM2', 'AZM2', 'ZM2', 'ELT', 'ZT', 'AZT', 'ELM1', 'AZM1', 'ZM1'])
    # Pickle the dataframe
    file = open(f'/scratch/labJackData/{expId}.pkl', 'wb')
    pkl.dump(df, file)
    file.close()
    finish = datetime.datetime.now()
    print(f"Finishing Accel data: {finish}")
    return 

async def waitForAccelData(readTime, expId):
    # This function wraps the readStream in an async function
    start = datetime.datetime.now()
    print(f"Starting Accel data: {start}")
    loop = asyncio.get_event_loop()
    await loop.run_in_executor(None, lambda:readStream(readTime, expId))

In [ ]:
# Now start up the observatory control stuff

In [ ]:
# for tab completion to work in current notebook instance
%config IPCompleter.use_jedi = False

In [ ]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG
# Make matplotlib less chatty
logging.getLogger("matplotlib").setLevel(logging.WARNING)

In [ ]:
# Set Cerro Pachon location
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [ ]:
#Start classes
domain = salobj.Domain()
await asyncio.sleep(10) # This can be removed in the future...
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

In [ ]:
# enable components
await atcs.enable({"atdome": "current", "ataos": "current", "athexapod": "current"})
await latiss.enable()

In [ ]:
async def objectExposure(expTime):
    # Will take exposures later
    # For now this is a dummy function
    start = datetime.datetime.now()
    print(f"Starting exposure: {start}")
    #await latiss.take_object(expTime, 1, filter='RG610', grating='empty_1')
    time.sleep(expTime) # take this out when taking actual exposure
    finish = datetime.datetime.now()
    print(f"Exposure done: {finish}")
    return

async def exposureWithAccelData(expTime, expId):
    # This puts them together
    readTime = expTime + 1.0 # Add some buffer for the accel data
    result = await asyncio.gather(waitForAccelData(readTime, expId), objectExposure(expTime))
    return

In [ ]:
# Take 10 biases to make sure things are working
# Added wait to stop killing the recent images
for i in range(10):
    await asyncio.sleep(2.0)
    await latiss.take_bias(1)

In [ ]:
# Set an AzEl location
start_az = 90.0
start_el = 45.0
start_rot = 0.0
await atcs.point_azel(start_az, start_el, rot_tel=start_rot)

In [ ]:
# get RA/DEC of current telescope Alt/Az position
az = Angle(start_az, unit=u.deg)
el = Angle(start_el, unit=u.deg)
print(f'orig az {az} and el {el}')
time_data = await atcs.rem.atptg.tel_timeAndDate.next(flush=True, timeout=2)

curr_time_atptg = Time(time_data.mjd, format="mjd")
coord_frame_AltAz = AltAz(location=location, obstime=curr_time_atptg)
coord_frame_radec = ICRS()
coord_azel = AltAz(az=az, alt=el, location=location, obstime=curr_time_atptg)
ra_dec = coord_azel.transform_to(coord_frame_radec)
print('Current Position is: \n {}'.format(coord_azel))
print('Current Position is: \n {}'.format(ra_dec))


In [ ]:
# Slew to starting position and start tracking
await atcs.slew_icrs(ra=str(ra_dec.ra), dec=str(ra_dec.dec), rot=0.0,
                      slew_timeout=240., stop_before_slew=False, wait_settle=False)


print('Tracking')



In [ ]:
# Now take an exposure
expTime = 5.0
expId = 2022030700002
await exposureWithAccelData(expTime, expId)

In [ ]:
await atcs.stop_tracking()

In [ ]:
# Now analyze it:
# Get EFD client and butler
client = EfdClient('summit_efd')
butler = Butler('/repo/LATISS', collections="LATISS/raw/all")

In [ ]:
# Here's a start on the analysis.  What needs to be done?
#     (1) Convert to Gen3 - done?
#     (2) Go to 4th order for fit - done?
#     (3) Add accel data to the graph - done?
async def MountTracking(expId, butler, client):
    # Find the time of exposure                                                                                
    mData = butler.get('raw.metadata', detector=0, exposure=expId)
    imgType = mData['IMGTYPE']
    tStart = mData['DATE-BEG']
    tEnd = mData['DATE-END']
    elevation = mData['ELSTART']
    azimuth = mData['AZSTART']
    print(f"expId = {expId}, imgType = {imgType}, Times = {tStart}, {tEnd}")
    if (imgType not in ['OBJECT', 'SKYEXP', 'ENGTEST', 'DARK']):
        return True
    end = time.time()
    elapsed = end-start
    print(f"Elapsed time for butler query = {elapsed}")
    start = time.time()

    # Get the data                                                                                             
    t_start = Time(tStart, scale='utc')
    t_end = Time(tEnd, scale='utc')

    az = await client.select_packed_time_series('lsst.sal.ATMCS.mount_AzEl_Encoders', \
                                                'azimuthCalculatedAngle', t_start, t_end)
    el = await client.select_packed_time_series('lsst.sal.ATMCS.mount_AzEl_Encoders', \
                                                'elevationCalculatedAngle', t_start, t_end)
    rot = await client.select_packed_time_series('lsst.sal.ATMCS.mount_Nasmyth_Encoders', \
                                                'nasmyth2CalculatedAngle', t_start, t_end)
    az_torque_1 = await client.select_packed_time_series('lsst.sal.ATMCS.measuredTorque', \
                                                'azimuthMotor1Torque', t_start, t_end)
    az_torque_2 = await client.select_packed_time_series('lsst.sal.ATMCS.measuredTorque', \
                                                'azimuthMotor2Torque', t_start, t_end)
    el_torque = await client.select_packed_time_series('lsst.sal.ATMCS.measuredTorque', \
                                                'elevationMotor2Torque', t_start, t_end)
    rot_torque = await client.select_packed_time_series('lsst.sal.ATMCS.measuredTorque', \
                                                'nasmyth2MotorTorque', t_start, t_end)

    end = time.time()
    elapsed = end-start
    print(f"Elapsed time to get the data = {elapsed}")
    start = time.time()

    # Calculate the tracking errors                                                                            
    az_vals = np.array(az.values.tolist())[:,0]
    el_vals = np.array(el.values.tolist())[:,0]
    rot_vals = np.array(rot.values.tolist())[:,0]
    times = np.array(az.values.tolist())[:,1]
    times = times - times[0]
    print("LengthAz", len(az_vals))
    # Fit with a quartic
    az_fit = np.polyfit(times, az_vals, 4)
    el_fit = np.polyfit(times, el_vals, 4)
    rot_fit = np.polyfit(times, rot_vals, 2)
    az_model = az_fit[0] * times * times * times * times + az_fit[1] * times * times * times \
    + az_fit[2] * times *times + az_fit[3] * times + az_fit[4]
    el_model = el_fit[0] * times * times * times * times + el_fit[1] * times * times * times \
    + el_fit[2] * times * times + el_fit[3] * times + el_fit[4]
    rot_model = rot_fit[0] * times * times + rot_fit[1] * times + rot_fit[2]

    # Errors in arcseconds                                                                                     
    az_error = (az_vals - az_model) * 3600
    el_error = (el_vals - el_model) * 3600
    rot_error = (rot_vals - rot_model) * 3600
    
    # Calculate RMS                                                                                            
    az_rms = np.sqrt(np.mean(az_error * az_error))
    el_rms = np.sqrt(np.mean(el_error * el_error))
    rot_rms = np.sqrt(np.mean(rot_error * rot_error))

    end = time.time()
    elapsed = end-start
    print(f"Elapsed time for error calculations = {elapsed}")
    start = time.time()
  
    # Unpickle the accel dataframe
    file = open(f'/scratch/labJackData/{expId}.pkl', 'rb')
    df = pkl.load(file)
    file.close()

    if makeGraph:

        fig = plt.figure(figsize = (16,16))
        plt.suptitle(f"Mount Tracking {expId}, Azimuth = {azimuth:.1f}, Elevation = {elevation:.1f}", fontsize\
 = 18)
        # Azimuth axis                                                                                         
        plt.subplot(4,3,1)
        ax1 = az['azimuthCalculatedAngle'].plot(legend=True, color='red')
        ax1.set_title("Azimuth axis", fontsize=16)
        ax1.axvline(tStart, color="red", linestyle="--")
        ax1.set_xticks([])
        ax1.set_ylabel("Degrees")
        plt.subplot(4,3,4)
        plt.plot(times, az_error, color='red')
        plt.title(f"Azimuth RMS error = {az_rms:.2f} arcseconds")
        plt.ylim(-10.0,10.0)
        plt.xticks([])
        plt.ylabel("ArcSeconds")
        plt.subplot(4,3,7)
        ax7 = az_torque_1['azimuthMotor1Torque'].plot(legend=True, color='blue')
        ax7 = az_torque_2['azimuthMotor2Torque'].plot(legend=True, color='green')
        ax7.axvline(tStart, color="red", linestyle="--")
        plt.subplot(4,3,10)
        ax10 = df['ELM2'].plot(color='red')
        #ax10.axvline(t_start.to_datetime(), color="blue", linestyle="--")
        #ax10.axvline(t_end.to_datetime(), color="blue", linestyle="--")
        ax10.set_xlim(t_start.to_datetime(), t_end.to_datetime())

        # Elevation axis                                                                                       
        plt.subplot(4,3,2)
        ax2 = el['elevationCalculatedAngle'].plot(legend=True, color='green')
        ax2.set_title("Elevation axis", fontsize=16)
        ax2.axvline(tStart, color="red", linestyle="--")
        ax2.set_xticks([])
        plt.subplot(4,3,5)
        plt.plot(times, el_error, color='green')
        plt.title(f"Elevation RMS error = {el_rms:.2f} arcseconds")
        plt.ylim(-10.0,10.0)
        plt.xticks([])
        plt.subplot(4,3,8)
        ax8 = el_torque['elevationMotorTorque'].plot(legend=True, color='blue')
        ax8.axvline(tStart, color="red", linestyle="--")
        plt.subplot(4,3,11)
        ax11 = df['AZM2'].plot(color='red')
        #ax11.axvline(t_start.to_datetime(), color="blue", linestyle="--")
        #ax11.axvline(t_end.to_datetime(), color="blue", linestyle="--")
        ax11.set_xlim(t_start.to_datetime(), t_end.to_datetime())
        
        # Nasmyth2 rotator axis                                                                                
        plt.subplot(4,3,3)
        ax3 = rot['nasmyth2CalculatedAngle'].plot(legend=True, color='blue')
        ax3.set_title("Nasmyth2 axis", fontsize=16)
        ax3.axvline(tStart, color="red", linestyle="--")
        ax3.set_xticks([])
        plt.subplot(4,3,6)
        plt.plot(times, rot_error, color='blue')
        plt.title(f"Nasmyth RMS error = {rot_rms:.2f} arcseconds")
        plt.ylim(-10000.0,10000.0)
        plt.subplot(4,3,9)
        ax9 = rot_torque['nasmyth2MotorTorque'].plot(legend=True, color='blue')
        ax9.axvline(tStart, color="red", linestyle="--")
        plt.savefig(f"/home/craiglagegit/DATA/mount_graphs/Mount_Accel_{expId}_08Mar22.pdf")                 

    end = time.time()
    elapsed = end-start
    print(f"Elapsed time for plots = {elapsed}")
    start = time.time()
    return
        

In [ ]:
# Done here.  The stuff below is just in case I need it.

In [ ]:
# Plot the data

sub_df = df[df.index[0] : df.index[999]]

plt.figure(figsize=(8,8))
plt.subplots_adjust(hspace=0.5, wspace=0.5)
plt.suptitle("Summit Container Test 07Mar22", fontsize=18)
plt.subplot(3,3,1)
ax1A = sub_df['ELM2'].plot(color='red')
ax1A.set_title("Elevation M2", fontsize=16)
ax1A.set_ylabel("Acceleration(g)")
plt.subplot(3,3,2)
ax1A = sub_df['ELT'].plot(color='red')
ax1A.set_title("Elevation Truss", fontsize=16)
ax1A.set_ylabel("Acceleration(g)")
plt.subplot(3,3,3)
ax1A = sub_df['ELM1'].plot(color='red')
ax1A.set_title("Elevation M1", fontsize=16)
ax1A.set_ylabel("Acceleration(g)")
plt.subplot(3,3,4)
ax1A = sub_df['AZM2'].plot(color='blue')
ax1A.set_title("Azimuth M2", fontsize=16)
ax1A.set_ylabel("Acceleration(g)")
plt.subplot(3,3,5)
ax1A = sub_df['AZT'].plot(color='blue')
ax1A.set_title("Azimuth Truss", fontsize=16)
ax1A.set_ylabel("Acceleration(g)")
plt.subplot(3,3,6)
ax1A = sub_df['AZM1'].plot(color='blue')
ax1A.set_title("Azimuth M1", fontsize=16)
ax1A.set_ylabel("Acceleration(g)")
plt.subplot(3,3,7)
ax1A = sub_df['ZM2'].plot(color='green')
ax1A.set_title("Optical-Axis M2", fontsize=16)
ax1A.set_ylabel("Acceleration(g)")
plt.subplot(3,3,8)
ax1A = sub_df['ZT'].plot(color='green')
ax1A.set_title("Optical-Axis Truss", fontsize=16)
ax1A.set_ylabel("Acceleration(g)")
plt.subplot(3,3,9)
ax1A = sub_df['ZM1'].plot(color='green')
ax1A.set_title("Optical-Axis M1", fontsize=16)
ax1A.set_ylabel("Acceleration(g)")

plt.savefig("/scratch/labJackData/Summit_Hack_Test_07Mar22.pdf")

In [ ]:
# Close handles
ljm.close(handle)

In [ ]:
# Unpickle the accel dataframe
file = open(f'/scratch/labJackData/{expId}.pkl', 'rb')
df = pkl.load(file)
file.close()


In [ ]:
#Starting exposure: 2022-03-07 16:43:21.691384
#Exposure done: 2022-03-07 16:43:25.695426
tstart = Time("2022-03-07T16:43:21.69", scale='utc')
tend = Time("2022-03-07T16:43:25.69", scale='utc')


In [ ]:
df.tail(1)

In [ ]:
ax1 = df['ELM2'].plot(color='red')
#ax1.axvline(tstart.to_datetime(), color="blue", linestyle="--")
#ax1.axvline(tend.to_datetime(), color="blue", linestyle="--")
ax1.set_xlim(tstart.to_datetime(), tend.to_datetime())
#ax1.set_ylim(0,1.0)